# Input data

In [4]:
# Import numpy and pandas
import numpy as np
import pandas as pd

# Load data from csv
housing_csv = pd.read_csv('datasets/kc_house_data.csv')

# Convert to numpy array
housing_array = np.array(housing_csv)

In [5]:
housing_csv.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [6]:
housing_array

array([[7129300520, '20141013T000000', 221900.0, ..., -122.257, 1340,
        5650],
       [6414100192, '20141209T000000', 538000.0, ..., -122.319, 1690,
        7639],
       [5631500400, '20150225T000000', 180000.0, ..., -122.233, 2720,
        8062],
       ...,
       [1523300141, '20140623T000000', 402101.0, ..., -122.299, 1020,
        2007],
       [291310100, '20150116T000000', 400000.0, ..., -122.069, 1410,
        1287],
       [1523300157, '20141015T000000', 325000.0, ..., -122.299, 1020,
        1357]], dtype=object)

In [7]:
# Import numpy and tensorflow with their standard aliases
import numpy as np
import tensorflow as tf

# Use a numpy array to define price as a 32-bit float
price = np.array(housing_csv['price'], np.float32)

# Define waterfront as a Boolean using cast
waterfront = tf.cast(housing_csv['waterfront'], tf.bool)

# Print price and waterfront
print(price)
print(waterfront)

[221900. 538000. 180000. ... 402101. 400000. 325000.]
tf.Tensor([False False False ... False False False], shape=(21613,), dtype=bool)


2023-05-20 11:20:25.815471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-20 11:20:25.815498: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-20 11:20:25.815518: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5855a770-b980-4a93-9254-7ae0a6563cbc): /proc/driver/nvidia/version does not exist
2023-05-20 11:20:25.815839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


**Note:** Notice that printing price yielded a numpy array; whereas printing waterfront yielded a tf.Tensor().

# Loss functions
- TensorFlow has operations for common loss functions
    - Mean squared error(MSE)
    - Mean absolute error(MAE)
    - Huber error

![image-3](image-3.png)


In [11]:
import tensorflow as tf
from tensorflow import Variable
from tensorflow import keras

# Initialize a variable named scalar
scalar = Variable(1.0, tf.float32)

# Instantiate features
features = Variable([1.0,2.0,3.0,4.0,5.0], tf.float32)

# Instantiate targets
targets = Variable([2.0,4.0,6.0,8.0,10.0], tf.float32)

# Define the model
def model(scalar, features = features):
  	return scalar * features

# Define a loss function
def loss_function(scalar, features = features, targets = targets):
	# Compute the predicted values
	predictions = model(scalar, features)
    
	# Return the mean absolute error loss
	return keras.losses.mae(targets, predictions)

# Evaluate the loss function and print the loss
print(loss_function(scalar).numpy())

3.0


# Linear regression in TensorFlow

In [22]:
# Define the targets and features
price = np.array(np.log(housing_csv['price']), np.float32)
size = np.array(np.log(housing_csv['sqft_living']),np.float32)

# Define the intercept and slope
intercept = tf.Variable(0.1, np.float32)
slope = tf.Variable(0.1, np.float32)

In [23]:
# Define a linear regression model
def linear_regression(intercept, slope, features =size):
    return intercept + features*slope

In [24]:
# Compute the predicted values and loss
def loss_function(intercept, slope, targets=price, features=size):
    predictions = linear_regression(intercept, slope)
    return tf.keras.losses.mse(targets, predictions)

In [25]:
# Compute the loss for different slope and intercept values
print(loss_function(0.1, 0.1).numpy())
print(loss_function(0.1, 0.5).numpy())

148.91295
84.309006


In [31]:
# Define an optimization operation - Adam with learning rate of 0.5
opt = tf.keras.optimizers.Adam(0.5)

In [36]:
intercept = tf.Variable(0.1)
slope = tf.Variable(0.5)

# Minimize the loss function and print the loss
for j in range(100): # Epoch=100
    opt.minimize(lambda: loss_function(intercept,slope),
                var_list=[intercept,slope])
    if j%10==0: # Print loss_function for every 10th epoch
        print(loss_function(intercept, slope))

tf.Tensor(11.748244, shape=(), dtype=float32)
tf.Tensor(0.3285229, shape=(), dtype=float32)
tf.Tensor(1.2011963, shape=(), dtype=float32)
tf.Tensor(1.9263566, shape=(), dtype=float32)
tf.Tensor(1.2263952, shape=(), dtype=float32)
tf.Tensor(0.5787599, shape=(), dtype=float32)
tf.Tensor(0.3191719, shape=(), dtype=float32)
tf.Tensor(0.2504808, shape=(), dtype=float32)
tf.Tensor(0.23634388, shape=(), dtype=float32)
tf.Tensor(0.23297228, shape=(), dtype=float32)


**Code Break-down**

- `opt.minimize`: This is the function call to an optimization method, which aims to minimize a given function.

- `lambda: loss_function(intercept, slope)`: This lambda function defines the loss function to be minimized. The lambda function takes no arguments (lambda:) and calls the loss_function with the intercept and slope parameters.

- `var_list=[intercept, slope]`: This parameter specifies the variables (intercept and slope) that will be optimized during the minimization process. The optimization algorithm will search for the optimal values of these variables that minimize the given loss function.

# Multiple linear regression with TensorFlow

In [42]:
# Initialize target and features
target = np.array(np.log(housing_csv['price'])) # Taking logarithmic for normaization
feature_1 = np.array(np.log(housing_csv['sqft_living'])) # Taking logarithmic for normaization
feature_2 = np.array(housing_csv['bedrooms'])

In [43]:
# Define multiple linear regression model
def mul_linear_regression(params, feature_1=feature_1, feature_2=feature_2):
    return params[0] + feature_1*params[1] + feature_2*params[2]

In [44]:
# Define loss function to calculate MAE
def loss_function(params, feature_1=feature_1, feature_2=feature_2, target=target):
    # Predict using multiple linear regression
    predictions = mul_linear_regression(params)
    
    # Calcuate MAE
    return keras.losses.mae(target, predictions)

In [51]:
# Loss function for params[0.1,0.02,0.5]
params = tf.Variable([0.1,0.02,0.5],tf.float32)
print(loss_function(params).numpy())

11.111702


In [52]:
# Optimizing using Adam optimizer
optim = tf.keras.optimizers.Adam()

In [58]:
# Minimize the loss funtion using optimizer and evaluate MAE
params = tf.Variable([0.1,0.02,0.5],tf.float32)
for j in range(100): #Epoch 100
    optim.minimize(lambda: loss_function(params), var_list=[params])
    # Print MAE based on given params for every 10th epoch
    if j%10 == 0:
        print("Params",params.numpy())
        print("MAE",loss_function(params).numpy())
        print()

Params [0.10198551 0.02198552 0.5019855 ]
MAE 11.08804

Params [0.13899067 0.05899072 0.53899074]
MAE 10.647036

Params [0.179733   0.09973308 0.57973313]
MAE 10.161495

Params [0.2168762  0.1368763  0.61687636]
MAE 9.718847

Params [0.25021484 0.17021497 0.650215  ]
MAE 9.321538

Params [0.28045553 0.20045567 0.6804556 ]
MAE 8.961149

Params [0.30827603 0.22827616 0.70827615]
MAE 8.629603

Params [0.33419386 0.254194   0.7341941 ]
MAE 8.320731

Params [0.358587   0.27858713 0.7585873 ]
MAE 8.030029

Params [0.38173223 0.30173236 0.7817325 ]
MAE 7.7542005



# Batch Training

In [82]:
# Define the intercept and slope
intercept = Variable(0.1, tf.float32)
slope = Variable(0.005, tf.float32)

# Define the model
def linear_regression(intercept, slope, features):
	# Define the predicted values
	return intercept + features*slope

# Define the loss function
def loss_function(intercept, slope, targets, features):
	# Define the predicted values
	predictions = linear_regression(intercept, slope, features)
    
 	# Define the MAE loss
	return keras.losses.mae(targets, predictions)

In [86]:
# Initialize Adam optimizer
opt = keras.optimizers.Adam()

# Count the batch
count = 0 

# Load data in batches
for batch in pd.read_csv('datasets/kc_house_data.csv', chunksize=100):
    
    # Extract sqft for current batch
    size_batch = np.array(batch['sqft_lot'], np.float32)

    # Extract the price values for the current batch
    price_batch = np.array(batch['price'], np.float32)
    
    # Batch count
    count += 1
    if count%10 == 0:
        print("MSE before minimization of ",count," batch:", loss_function(intercept,
                                                            slope, 
                                                            price_batch, 
                                                            size_batch).numpy())
    
    # Minimization of parameters
    opt.minimize(lambda: loss_function(intercept, slope, price_batch, size_batch), 
                 var_list=[intercept, slope])
    
    if count%10 == 0:
        print("MSE after minimization of ",count, " batch:", loss_function(intercept,
                                                                       slope, 
                                                                       price_batch, 
                                                                       size_batch).numpy())
        print()
    

MSE before minimization of  10  batch: 466823.28
MSE after minimization of  10  batch: 466811.62

MSE before minimization of  20  batch: 507766.3
MSE after minimization of  20  batch: 507753.72

MSE before minimization of  30  batch: 509910.75
MSE after minimization of  30  batch: 509888.2

MSE before minimization of  40  batch: 589219.8
MSE after minimization of  40  batch: 589193.2

MSE before minimization of  50  batch: 550550.56
MSE after minimization of  50  batch: 550534.2

MSE before minimization of  60  batch: 528731.75
MSE after minimization of  60  batch: 528717.5

MSE before minimization of  70  batch: 521815.1
MSE after minimization of  70  batch: 521798.7

MSE before minimization of  80  batch: 546294.94
MSE after minimization of  80  batch: 546284.7

MSE before minimization of  90  batch: 509868.6
MSE after minimization of  90  batch: 509855.88

MSE before minimization of  100  batch: 524421.9
MSE after minimization of  100  batch: 524409.06

MSE before minimization of  1

Now while minimizing dataset, we have around 210 iterations because of the chunksize of 100, at every iteration, 100 datapoints are used to minimize. Thus minimization occurs around 210 times for each epoch. Hence, batch training is more powerful. 